In [2]:
import pandas as pd
import numpy as np
import os 
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:

bookReviewDataSet_filename = os.path.join(os.getcwd(), "data", "bookReviewsData.csv")


df = pd.read_csv(bookReviewDataSet_filename)

In [4]:
df.head()
df.shape
print(df['Review'])
df.isnull().values.any()

0       This was perhaps the best of Johannes Steinhof...
1       This very fascinating book is a story written ...
2       The four tales in this collection are beautifu...
3       The book contained more profanity than I expec...
4       We have now entered a second time of deep conc...
                              ...                        
1968    I purchased the book with the intention of tea...
1969    There are so many design books, but the Graphi...
1970    I am thilled to see this book being available ...
1971    As many have stated before me the book starts ...
1972    I love this book! It is a terrific blend of ha...
Name: Review, Length: 1973, dtype: object


False

# Project Plan

For this project, df['Review'] will be the feature and a new column df['Genre'] will be my label. Creating a dictionary of common words associated with select genres, I will go through each review and assign it a genre based on which words it included the most. I will follow the NLP pipeline by first tokenizing and preprocessing my text before performing TF-IDF tokenization. I will be training a Logistic Regression, Random Forest, GBDT, and Decision Tree model. I will be performing a grid search to find the best hyperparameters from a parameter grid I will create. In the event that my data is imbalanced, in addition to the accuracy scores, I will find the balanced_accuracy_score. 

In [5]:
!pip install gensim
!pip install nltk

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import gensim
import nltk
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_curve
from sklearn.metrics import balanced_accuracy_score


[notice] A new release of pip is available: 23.0.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [6]:
#First, perform tokenization and processing


#simple_preporcess removes some stop words, converts all text to lowercase, 
#removes punctuation, and tokenizes the text
df['Review']=df['Review'].apply(lambda row: gensim.utils.simple_preprocess(row))


genre_words={
    'romance': ['romance','affair','affection','heart','passion'],
    'sci-fi': ['science fiction','space', 'science','alien','sci-fi'],
    'historical': ['history', 'historical','past','time','period','war'],
    'fantasy': ['wizard', 'fairy','magic','magical','fantasy','dragon'],
    'children': ['children','child','baby','kid','education','educational'],
    'thriller': ['thriller','suspense','suspensful','tense','danger','unexpected','twist','hawking'],
    'adventure': ['journey','adventure', 'hero','voyage','expedition']
}

def genre_finder(tokens, genre_words):
    word_count=defaultdict(int)
    for genre, keywords in genre_words.items():
        for k in keywords:
            word_count[genre]+=tokens.count(k)
            
    return max(word_count, key=word_count.get)


df['Genre']=df['Review'].apply(lambda rows: genre_finder(rows, genre_words))

remove_rows=df.isnull().values.any()
if(remove_rows==True):
    df=df.dropna()
    

In [7]:
df['Genre'].value_counts()

romance       1128
historical     539
children       137
sci-fi          72
adventure       39
thriller        32
fantasy         26
Name: Genre, dtype: int64

In [8]:
#start modeling process
y=df['Genre']
X=df['Review']
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.25, random_state=1234)


def tokenized(tokens):
    return tokens
#Implementing TF-IDF Vectorizer to Transform Text
tfidf_vectorizer=TfidfVectorizer(analyzer='word', tokenizer=tokenized, preprocessor=None, lowercase=False )
tfidf_vectorizer.fit(X_train)
X_train=tfidf_vectorizer.transform(X_train)
X_test=tfidf_vectorizer.transform(X_test)
#After this, X_train changes from an array of text reviews
#to a sparse matrix of TF-IDF features


/home/ubuntu/.pyenv/versions/3.9.19/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [9]:
#develop models
lr_cvalues=[10**i for i in range(-5, 5)]
models={}
model_scores={}
model_acc_scores={}
model_predictions={}
imbalanced_acc_scores={}
model_names={
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Random Forest': RandomForestClassifier(),
    'Decision Tree': DecisionTreeClassifier(),
    'GBDT': GradientBoostingClassifier()}

param_grids={
    'Logistic Regression': {'C': lr_cvalues},
    'Random Forest':  {'max_depth': [10, 20, 30], 'n_estimators': [10,20,30,40]}    ,
    'Decision Tree':{ 'max_depth':[4,8,12,32] ,'min_samples_leaf':[25,50,75]   }  ,
    'GBDT': { 'max_depth':[2,4], 'n_estimators':  [100,150] }   
            }

for name, model in model_names.items():
    print("performing grid search")
    grid = GridSearchCV(model, param_grids[name], cv=3,scoring='accuracy')
    grid_search=grid.fit(X_train, y_train)
    models[name]=grid_search.best_estimator_
    model_scores[name]=grid_search.best_score_

    label_prediction=models[name].predict(X_test)
    model_predictions[name]=label_prediction
    
    model_acc=accuracy_score(y_test, label_prediction)
    imbalanced_acc=balanced_accuracy_score(y_test, label_prediction)
    #accuracy_score evaluates final model performance
    model_acc_scores[name]=model_acc
    imbalanced_acc_scores[name]=imbalanced_acc
    print(model_acc_scores[name])
best_model=max(model_acc_scores, key=model_acc_scores.get)
best_balanced_model=max(imbalanced_acc_scores, key=imbalanced_acc_scores.get)




performing grid search
0.7854251012145749
performing grid search
0.7186234817813765
performing grid search
0.840080971659919
performing grid search
0.9392712550607287


In [10]:

print(f'The model with the highest accuracy score is: {best_model} with a score of: { model_acc_scores[best_model] }')
print(f'The model with the highest balanced accuracy score is: {best_balanced_model} with a score of:{ imbalanced_acc_scores[best_balanced_model] } ')
#Here, I wanted to take a glance at the actual vs. predicted genres of the best-performing model
for best_model, p in model_predictions.items():
    df_gbdt=pd.DataFrame({
        'Review': 'Review',
        'Actual_genre': y_test,
        'Predicted_genre': p
    })
print(df_gbdt)

#The balanced accuracy scores of all the models
for m, score in imbalanced_acc_scores.items():
    print(score)

The model with the highest accuracy score is: GBDT with a score of: 0.9392712550607287
The model with the highest balanced accuracy score is: GBDT with a score of:0.7887588712715764 
      Review Actual_genre Predicted_genre
1692  Review     children          sci-fi
1744  Review    adventure       adventure
1236  Review      romance         romance
21    Review      romance         romance
894   Review      romance         romance
...      ...          ...             ...
1569  Review      romance         romance
256   Review   historical      historical
1969  Review      romance         romance
1188  Review   historical      historical
817   Review      romance         romance

[494 rows x 3 columns]
0.43510936331464933
0.29089202967067107
0.38928475828171605
0.7887588712715764
